In [2]:
import json
import pandas as pd
import numpy as np
import re
# import nltk
# from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [4]:
with open('dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)
print(df.head())

                                               title  \
0  Cá nhân hóa trải nghiệm bằng công nghệ tiên ti...   
1                       Microsoft ra mắt Office 2024   
2  Những thành tựu khó đạt được nhất trong Black ...   
3  Tựa game "đắt" nhất trên Steam, mua đủ hết nội...   
4  CEO Game Science trả lời hài hước về khả năng ...   

                                            subtitle    category          time  
0  Chính thức hoạt động từ cuối tháng 7/2024, Ngâ...  Apps/Games  5 ngày trước  
1  Office 2024 đã chính thức ra mắt với hàng loạt...  Apps/Games  5 ngày trước  
2  Nếu như không có sự trợ giúp từ Internet, các ...  Apps/Games  8 ngày trước  
3  Thế nhưng đa phần các game thủ Steam đều chẳng...  Apps/Games  8 ngày trước  
4          Trong lúc chờ, bạn hãy chơi game khác đi!  Apps/Games  9 ngày trước  


In [5]:
tmp_label = ['Apps/Games', 'Internet', 'Khám phá', 'Mobile', 'Tin ICT', 'Đồ chơi số']
df = df[df['category'].isin(tmp_label)]
df['text'] = df['title'] + " " + df['subtitle']
df = df[['text', 'category']]
print(df.head())

                                                text    category
0  Cá nhân hóa trải nghiệm bằng công nghệ tiên ti...  Apps/Games
1  Microsoft ra mắt Office 2024 Office 2024 đã ch...  Apps/Games
2  Những thành tựu khó đạt được nhất trong Black ...  Apps/Games
3  Tựa game "đắt" nhất trên Steam, mua đủ hết nội...  Apps/Games
4  CEO Game Science trả lời hài hước về khả năng ...  Apps/Games


In [6]:
# nltk.download('stopwords')
# stop_words = set(stopwords.words('vietnamese'))
stop_words = []

In [7]:
def clean_text(text):
    # Chuyển văn bản về chữ thường
    text = text.lower()
    
    # Loại bỏ ký tự đặc biệt và dấu câu
    text = re.sub(r'\W', ' ', text)
    
    # Loại bỏ các số
    text = re.sub(r'\d+', '', text)
    
    # Loại bỏ các từ dừng (stopwords)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text

In [8]:
df['text'] = df['text'].apply(clean_text)
print(df)

                                                    text    category
0      cá nhân hóa trải nghiệm bằng công nghệ tiên ti...  Apps/Games
1      microsoft ra mắt office office đã chính thức r...  Apps/Games
2      những thành tựu khó đạt được nhất trong black ...  Apps/Games
3      tựa game đắt nhất trên steam mua đủ hết nội du...  Apps/Games
4      ceo game science trả lời hài hước về khả năng ...  Apps/Games
...                                                  ...         ...
11986  đóng loạt cửa hàng doanh thu thế giới di động ...     Tin ICT
11987  tử huyệt ô tô điện xe cũ mất giá quá nhanh hàn...     Tin ICT
11988  samsung hưởng lợi lớn từ lệnh cấm bán apple wa...     Tin ICT
11989  năm việt nam sẽ có phần mềm chống lừa đảo trực...     Tin ICT
11990  twitter quỵt hàng triệu usd tiền thưởng của nh...     Tin ICT

[10645 rows x 2 columns]


In [11]:
# Chuyển đổi văn bản thành ma trận TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)  # Giới hạn số lượng đặc trưng
X = vectorizer.fit_transform(df['text']).toarray()

print(X[100])

# Chuyển nhãn (category) thành dạng số
le = LabelEncoder()
y = le.fit_transform(df['category'])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.20945499 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.12055514 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [102]:
print(np.unique(df['category']))
print(X.shape)
print(y.shape)

['Apps/Games' 'Internet' 'Khám phá' 'Mobile' 'Tin ICT' 'Đồ chơi số']
(10645, 1000)
(10645,)


# Split Dataset

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train

In [104]:
model = LogisticRegression()
model.fit(X_train, y_train)

c:\Users\nquoc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

# Predict

In [105]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.6824800375763269
              precision    recall  f1-score   support

  Apps/Games       0.69      0.75      0.72       337
    Internet       0.59      0.56      0.57       409
    Khám phá       0.82      0.67      0.74       192
      Mobile       0.83      0.81      0.82       397
     Tin ICT       0.52      0.55      0.53       407
  Đồ chơi số       0.74      0.78      0.76       387

    accuracy                           0.68      2129
   macro avg       0.70      0.68      0.69      2129
weighted avg       0.69      0.68      0.68      2129



# -------

In [106]:
new_text = "Apple ra mắt sản phẩm mới MacBook Pro 2024 với thiết kế mỏng hơn"
new_text_clean = clean_text(new_text)

# Chuyển đổi thành vector TF-IDF
new_text_vector = vectorizer.transform([new_text_clean])

predicted_category = le.inverse_transform(model.predict(new_text_vector))

print(f'Predicted Category: {predicted_category[0]}')


Predicted Category: Đồ chơi số
